In [1]:
import app

/tmp/ipykernel_24995/3720885069.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  import app


In [ ]:
from app import graph
app.invoke({"message":["hi"]})

AttributeError: module 'app' has no attribute 'invoke'

In [1]:
from typing import List, Annotated, TypedDict
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool

# Your local modules
from research.helpers import LLM
from research.tools import pandas_tool, sql_tool, rag_tool

# -------------------------
# Graph state
# -------------------------
class GraphState(TypedDict, total=False):
    messages: Annotated[List[BaseMessage], add_messages]
    inputs: str

# -------------------------
# Wrap existing functions as LangChain Tools
# (names can be anything, but they must match what the LLM may call)
# -------------------------
@tool
def pandas_exec(query: str) -> str:
    """Run a pandas task on in-memory data as described by `query`."""
    return str(pandas_tool.pandas_tool(query))

@tool
def sql_exec(query: str) -> str:
    """Run a SQL/pandasql task described by `query`."""
    return str(sql_tool.pandasql_tool(query))

@tool
def rag_exec(query: str) -> str:
    """Run a retrieval-augmented lookup to answer `query`."""
    return str(rag_tool.rag_tool(query))

TOOLS = [pandas_exec, sql_exec, rag_exec]

# -------------------------
# Build the graph
# -------------------------
def create_graph():
    builder = StateGraph(GraphState)

    # 1) Agent node: call model *bound to tools* so it can emit tool_calls
    agent_llm = LLM().llm.bind_tools(TOOLS)

    def agent_node(state: GraphState) -> GraphState:
        msgs = state.get("messages", [])
        # Ensure the latest user input is present as a HumanMessage
        if not msgs or not isinstance(msgs[-1], HumanMessage):
            msgs = msgs + [HumanMessage(content=state["inputs"])]

        ai = agent_llm.invoke(msgs)   # may include tool_calls
        return {"messages": [ai]}

    # 2) Tools node: executes tool_calls found in the last AI message
    tools_node = ToolNode(TOOLS)

    # 3) Register nodes
    builder.add_node("agent", agent_node)
    builder.add_node("tools", tools_node)

    # 4) Wire edges
    builder.add_edge(START, "agent")

    # If the LLM requested a tool, go to ToolNode; else, END.
    # path_map keeps the diagram clean and avoids stray auto-edges in some versions.
    builder.add_conditional_edges("agent", tools_condition, path_map=["tools", END])

    # After running a tool, go back to the agent to continue or finish
    builder.add_edge("tools", "agent")

    return builder.compile()


/workspaces/new_rag/research/tools/pandas_tool.py:10: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .state import GraphState


In [2]:
app=create_graph()
png_bytes = app.get_graph().draw_mermaid_png(output_file_path="graph.png")

In [2]:
from typing import List, Annotated, TypedDict
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate

# Your local modules
from research.helpers import LLM
from research.tools import pandas_tool, sql_tool, rag_tool
class GraphState(TypedDict, total=False):
    messages: Annotated[List[BaseMessage], add_messages]
    inputs: str
    choice: str  # Directly store the tool choice as a string


# -------------------------
# Graph builder
# -------------------------
def create_graph():
    graph = StateGraph(GraphState)

    # 1) Input node: decide which tool to use by calling the LLM
    def inputs_node(state: GraphState) -> GraphState:
        state["inputs"] = input("Enter your query: ")
        user_input = state["inputs"]

        prompt = PromptTemplate(
            template="""You are a router that decides which tool to use for a user request.

User input:
{user_input}

Return one of:
- "pandas" for DataFrame wrangling/visualizations
- "sql" for queries that should be answered with SQL (incl. pandasql)
- "rag" for retrieval-augmented generation (document/knowledge lookup)

Return ONLY the literal label.""",
inputs=["user_input"]
        )
        prompt=prompt.format(user_input=user_input)

        response = LLM().llm.invoke({"user_input": user_input, "prompt": prompt})

        # Directly store the choice as a string
        state["choice"] = response  # Assuming response is a string

        return state


/workspaces/new_rag/research/tools/pandas_tool.py:10: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from .state import GraphState


In [8]:
from langchain_core.prompts import PromptTemplate
from typing import List, Annotated, TypedDict
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate

# Your local modules
from research.helpers import LLM
from research.tools import pandas_tool, sql_tool, rag_tool

def inputs_node():
        user_input  = input("Enter your query: ")
        

        prompt = PromptTemplate(
            template="""You are a router that decides which tool to use for a user request.

User input:
{user_input}

Return one of:
- "pandas" for DataFrame wrangling/visualizations
- "sql" for queries that should be answered with SQL (incl. pandasql)
- "rag" for retrieval-augmented generation (document/knowledge lookup)

Return ONLY the literal label.""",
inputs=["user_input"]
        )
        prompt=prompt.format(user_input=user_input)

        response = LLM().llm.invoke(prompt)

        # Directly store the choice as a string
        return response  # Assuming response is a string

In [9]:
x=inputs_node()

In [11]:
x.content

'pandas'

In [2]:
tools = [pandas_tool.pandas_tool, sql_tool.pandasql_tool, rag_tool.rag_tool]


In [7]:
tools[0].invoke({"state":state})

ValidationError: 2 validation errors for pandas_tool
state.answer
  Field required [type=missing, input_value={'messages': [HumanMessag...hi', 'choice': 'pandas'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
state.tool_choice
  Field required [type=missing, input_value={'messages': [HumanMessag...hi', 'choice': 'pandas'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [6]:
state={"messages":[HumanMessage(content="hi")],"inputs":"hi","choice":"pandas"}